In [1]:
import pandas as pd
import numpy as np
import io
import os
import json
import distutils.dir_util
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

## 데이터 전처리

In [2]:
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
# train data 준비
f_train = pd.read_json('../kakao_arena_melon/arena_data/orig/train.json' ,typ = 'frame', encoding='utf-8')
df_train = pd.DataFrame(f_train)
df_train = df_train.sort_values(by=['like_cnt'],ascending=False)
df_train = df_train[df_train['like_cnt']>5]
df_train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
8699,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 368264, 600999, 274905, 643023, 24070...",53211,2020-04-17 14:32:23.000
61103,"[감성, 새벽, 때껄룩, 잔잔한, 휴식, 팝송, 떼껄룩, 힐링, 팝, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 432753, 176384, 152694, 60126...",41844,2020-04-23 20:58:45.000
19863,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 256568, 7...",27268,2020-04-19 21:22:11.000


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53186 entries, 8699 to 31216
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          53186 non-null  object
 1   id            53186 non-null  int64 
 2   plylst_title  53186 non-null  object
 3   songs         53186 non-null  object
 4   like_cnt      53186 non-null  int64 
 5   updt_date     53186 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [4]:
# 플레이리스트 내 song id 리스트
train_song = df_train['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_train['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [5]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [6]:
count_song

Counter({159317: 3,
         368264: 12,
         600999: 3,
         274905: 6,
         643023: 47,
         240703: 3,
         452196: 32,
         424328: 41,
         380410: 38,
         280877: 4,
         248200: 29,
         204192: 16,
         260161: 3,
         248316: 4,
         520177: 7,
         388089: 3,
         409625: 6,
         274719: 3,
         570357: 2,
         258956: 9,
         342116: 8,
         587463: 1,
         438327: 13,
         25767: 1,
         481009: 16,
         529425: 1,
         621033: 28,
         518481: 3,
         96298: 1,
         382435: 9,
         649213: 2,
         212859: 10,
         37254: 13,
         526953: 4,
         502224: 3,
         585597: 2,
         144941: 9,
         463850: 3,
         217952: 5,
         285826: 2,
         431974: 4,
         488837: 10,
         208068: 2,
         336281: 1,
         238669: 2,
         94648: 11,
         601447: 4,
         34552: 3,
         384449: 11,
         7

In [7]:
# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 이상 담긴 song의 개수
song_len

24537

In [8]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 11회 이상 담긴 tag의 개수
tag_len

1619

In [9]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

26156

In [102]:
results = pd.read_json('C:/ROOT_final/final_project/autoencoder_denoise/results_denoise_ad.json',typ = 'frame', encoding='utf-8', )
results.head(3)

,id,songs,tags
0,102123,"[193147, 701408, 287328, 666852, 335520, 40560...","[사랑, 잔잔한, 연애, 발랄, 연애세포, 행복, 설렘, 두근두근, 달달, 봄]"
1,11762,"[188716, 555338, 622717, 413444, 542045, 41276...","[취향저격, 공부, 자기전에, 혼자, 위로, 휴식, 잠들기전, 감성, 새벽, 잔잔한]"
2,56212,"[495557, 486862, 201175, 205757, 90675, 152269...","[JAYJE, Pop, 낭만, 학창시절, 2000년대, 팝, 소울, 팝음악, 추위,..."


In [103]:
pl_ids = results['id']
len(pl_ids)

23015

In [104]:
p_songs = results['songs']
len(p_songs)

23015

In [105]:
p_tags = results['tags']
len(p_songs)

23015

In [106]:
p_songs_reranked = []
for p_song in p_songs :
    score = []
    for song in p_song :
        score.append(count_song[song])
    p_song_reranked = np.array(p_song)[np.array(score).argsort()].tolist()
    p_songs_reranked.append(p_song_reranked)
len(p_songs_reranked)


23015

In [107]:
p_tags_reranked = []
for p_tag in p_tags :
    score = []
    for tag in p_tag :
        score.append(count_tag[tag])
    p_tag_reranked = np.array(p_tag)[np.array(score).argsort()].tolist()
    p_tags_reranked.append(p_tag_reranked)
len(p_tags_reranked)

23015

In [108]:
result=[]
for i in range(len(pl_ids)) :
    dic={}
    dic['id']=pl_ids[i]
    dic['songs']=p_songs_reranked[i]
    dic['tags']=p_tags_reranked[i]
    result.append(dic)

In [109]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath(parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [110]:
write_json(result,'results_denoise_ad_rrk')